<a href="https://colab.research.google.com/github/gacerioni/Redis-VectorDB-LLM-RAG-Workshop/blob/main/03-Advanced_RedisSearch/03-Advanced_RedisSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RediSearch

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

Este notebook é uma versão adaptada e simplificada da documentação de quickstart do RedisInsight. E vamos trabalhar com Hashes aqui; por opção mesmo.

In [1]:
# Vamos instalar a lib do redis localmente aqui no Colab
!pip install -q redis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 3.5 MB/s eta 0:00:00


In [3]:
# E o RedisStack, como sempre fazemos
# Pode pular caso ja esteja no Redis Cloud
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


In [4]:
# Novamente, configurando o client e fazendo um teste basico
import os

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

#shortcut for redis-cli $REDIS_CONN command
# If SSL is enabled on the endpoint add --tls
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Test Redis connection
!redis-cli $REDIS_CONN PING

PONG


In [5]:
import redis
r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
r.ping()

True

## Redis Hashes


O Hash, no Redis, é um tipo de dado fundamental. Agora o JSON também foi incorporado, mas o Hash faz parte da stack inicial, old-school, e é tão legal quando JSON.


Para um deep-dive nos HASH commands, veja aqui: https://redis.io/commands/?group=hash




In [7]:
schools = [
    {"name":"Hall School",
     "description":"Spanning 10 states, this school award-winning curriculum includes a comprehensive reading system (from letter recognition and phonics to reading full-length books), as well as math, science, social studies, and even  philosophy.",
     "class":"independent",
     "type":"traditional",
     "address_city":"London",
     "address_street":"Manor Street",
     "students":342,
     "location":"51.445417, -0.258352"
     },
    {"name":"Garden School",
     "description":"Garden School is a new and innovative outdoor teaching and learning experience, offering rich and varied activities in a natural environment to children and families.",
     "class":"state","type":"forest; montessori",
     "address_city":"London",
     "address_street":"Gordon Street",
     "students":1452,
     "location":"51.402926, -0.321523",
     },

    {"name":"Gillford School",
     "description":"Gillford School is an inclusive learning centre welcoming people from all walks of life, here invited to step into their role as regenerative agents, creating new pathways into the future and inciting an international movement of cultural, land, and social transformation.",
     "class":"private",
     "type":"democratic; waldorf",
     "address_city":"Goudhurst",
     "address_street":"Goudhurst",
     "students":721,
     "location":"51.112685, 0.451076"
     },

     {
     "name":"Forest School",
     "description":"The philosophy behind Forest School is based upon the desire to provide young children with an education that encourages appreciation of the wide world in nature while achieving independence, confidence and high self-esteem. ",
     "class":"independent",
     "type":"forest; montessori; democratic",
     "address_city":"Oxford",
     "address_street":"Trident Street",
     "students":1200,
     "location":"51.781756, -1.123196"
     }
    ]
# carregue os dados no Redis as HASH agora
for id,school in enumerate(schools):
    #print(school)
    r.hset(f"school:{id}", mapping = school)

In [8]:
!redis-cli $REDIS_CONN keys 'school:*'
!echo
!redis-cli $REDIS_CONN HGETALL school:1



1) "school:3"
2) "school:1"
3) "school:0"
4) "school:2"

 1) "address_city"
 2) "London"
 3) "class"
 4) "state"
 5) "address_street"
 6) "Gordon Street"
 7) "name"
 8) "Garden School"
 9) "type"
10) "forest; montessori"
11) "description"
12) "Garden School is a new and innovative outdoor teaching and learning experience, offering rich and varied activities in a natural environment to children and families."
13) "students"
14) "1452"
15) "location"
16) "51.402926, -0.321523"


In [ ]:
#!redis-cli $REDIS_CONN flushdb

## RediSearch

**RediSearch** adiciona a capacidade de consultar dados em suas estruturas de dados **HASH** ou **JSON**, essencialmente transformando o Redis em um banco de dados de documentos.

Com o RediSearch, você declara índices uma vez e, em seguida, cada objeto de banco de dados que corresponder ao prefixo definido no índice será automaticamente e em tempo real adicionado ao índice.

Para a lista completa de comandos do RediSearch, veja: https://redis.io/commands/?group=search

Documentação Python: https://redis-py.readthedocs.io/en/stable/redismodules.html#redisearch-commands


-----

Aqui, vamos começar a brincar com o Index.\
Note que temos um campo do tipo `GeoField`. Vamos usar ele muito em breve!

In [9]:
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    GeoField,
    VectorField
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from redis.commands.search.aggregation import AggregateRequest
from redis.commands.search import reducers
schema = (
    TextField("name", as_name="name"),
    TextField("description", as_name="description"),
    TagField("address_city", as_name="city"),
    TagField("type", as_name="type", separator=";"),
    NumericField("students", as_name="students"),
    GeoField("location", as_name="location")
    )
r.ft("idx:schools").create_index(schema,
                    definition=IndexDefinition(prefix=["school:"],
                    index_type=IndexType.HASH)
                    )

b'OK'

In [11]:
# Vamos pegar as infos do indice. Tem coisa legal ai, viu.
r.ft("idx:schools").info()


{'index_name': 'idx:schools',
 'index_options': [],
 'index_definition': [b'key_type',
  b'HASH',
  b'prefixes',
  [b'school:'],
  b'default_score',
  b'1'],
 'attributes': [[b'identifier',
   b'name',
   b'attribute',
   b'name',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'description',
   b'attribute',
   b'description',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'address_city',
   b'attribute',
   b'city',
   b'type',
   b'TAG',
   b'SEPARATOR',
   b','],
  [b'identifier',
   b'type',
   b'attribute',
   b'type',
   b'type',
   b'TAG',
   b'SEPARATOR',
   b';'],
  [b'identifier', b'students', b'attribute', b'students', b'type', b'NUMERIC'],
  [b'identifier', b'location', b'attribute', b'location', b'type', b'GEO']],
 'num_docs': '4',
 'max_doc_id': '4',
 'num_terms': '130',
 'num_records': '158',
 'inverted_sz_mb': '8.802413940429688e-4',
 'vector_index_sz_mb': '0',
 'total_inverted_index_blocks': '140',
 'offset_vectors_sz_mb': 

Finalmente, vamos fazer umas buscas legais, usando o Index que criamos aqui acima.

Primeiro, vamos criar algumas functions de apoio para a demo:

In [13]:
import pandas as pd

# helper function para mostrar os resultados do redis.ft().search() como um dataframe, pra quem curte pandas
def display_ft(res):
  if res.total==0:
    print("No matches found")
  else:
    res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
    display(res_df)

# helper function para traduzir o resultado do FT.AGGREGATE para um dataframe e entao mostrar o resultado
def display_ft_agg(res):
  data = res.rows
  data = [[item.decode('utf-8') for item in sublist] for sublist in data]
  column_dict = {}
  for sublist in data:
      for i in range(0, len(sublist), 2):
          column_name = sublist[i]
          column_value = sublist[i + 1]
          column_dict.setdefault(column_name, []).append(column_value)
  df = pd.DataFrame(column_dict)
  display(df)

## Text search

Você pode executar consultas de pesquisa de texto (FT Search) em qualquer campo que você marcou para ser indexado como `TEXT` ou `TextField()` em Python.

Para especificar o campo específico, use a sintaxe `@field:value`.

Você também pode fazer correspondência de padrões, usando `@field:val*`

Observe que não estamos falando de TAGs, mas TEXT agora.

In [14]:
# retorne o documento inteiro, como fizemos com JSON
res=r.ft("idx:schools").search("nature")
display_ft(res)

,id,address_city,class,students,address_street,type,description,name,location
0,school:3,Oxford,independent,1200,Trident Street,forest; montessori; democratic,The philosophy behind Forest School is based u...,Forest School,"51.781756, -1.123196"
1,school:1,London,state,1452,Gordon Street,forest; montessori,Garden School is a new and innovative outdoor ...,Garden School,"51.402926, -0.321523"


In [15]:
# Full text search, voltando apenas alguns campos
query=Query("nature") \
   .return_field("address_city", as_field="city") \
   .return_field("name", as_field="name")
res=r.ft("idx:schools").search(query)
display_ft(res)


,id,city,name
0,school:3,Oxford,Forest School
1,school:1,London,Garden School


## Avançando - Busca com múltiplos parametros

Você pode combinar condições de busca em múltiplos "campos" do index, usando operadores lógicos:
  - Usando um espaço entre as condições, você terá um `AND`;
  - Usando um pipe `|`, você terá um `OR`.

In [16]:
# Rode uma busca pelos docs que possuem TODAS as Tags (AND condition)
query=Query("@type:{forest} @type:{montessori}")
res=r.ft("idx:schools").search(query)
display_ft(res)


,id,address_city,class,students,address_street,type,description,name,location
0,school:3,Oxford,independent,1200,Trident Street,forest; montessori; democratic,The philosophy behind Forest School is based u...,Forest School,"51.781756, -1.123196"
1,school:1,London,state,1452,Gordon Street,forest; montessori,Garden School is a new and innovative outdoor ...,Garden School,"51.402926, -0.321523"


In [17]:
# Rode uma busca pelos docs que estao em Goudhurst OU tenha o tipo montessori (OR condition)
query=Query("(@city:{Goudhurst})|(@type:{montessori})")
res=r.ft("idx:schools").search(query)
display_ft(res)

,id,address_city,class,students,address_street,type,description,name,location
0,school:2,Goudhurst,private,721,Goudhurst,democratic; waldorf,Gillford School is an inclusive learning centr...,Gillford School,"51.112685, 0.451076"
1,school:3,Oxford,independent,1200,Trident Street,forest; montessori; democratic,The philosophy behind Forest School is based u...,Forest School,"51.781756, -1.123196"
2,school:1,London,state,1452,Gordon Street,forest; montessori,Garden School is a new and innovative outdoor ...,Garden School,"51.402926, -0.321523"


## TAG, Numeric and Geo search

For TAG fields - use `@field:{value}` syntax.

Geo radius matches on geo fields with the syntax `@field:[lon lat radius {m|km|mi|ft}]`

Numeric ranges look like `@students:[0, 10000]` with square brackets used as inclusive (less or equal, greater or equal) and regular braces for exclusive (less or greater).

In [ ]:
#Geolocation search
query = Query('@location:[51.3 0.32 30 km]')
res = r.ft('idx:schools').search(query)
display_ft(res)

,id,students,address_street,type,location,address_city,name,class,description
0,school:2,721,Goudhurst,democratic; waldorf,"51.112685, 0.451076",Goudhurst,Gillford School,private,Gillford School is an inclusive learning centr...


In [ ]:
#Combining Tag and Geo conditions
query=Query('@city:{London} @students:[0, 10000]') \
   .return_field("address_city", as_field="city") \
   .return_field("name", as_field="name") \
   .return_field("students", as_field="students")
res=r.ft("idx:schools").search(query)
display_ft(res)



,id,city,name,students
0,school:1,London,Garden School,1452
1,school:0,London,Hall School,342


## Aggregations
Aggregations are a way to process the results of a search query, group, sort and transform them - and extract analytic insights from them. Much like aggregation queries in other databases and search engines, they can be used to create analytics reports, or perform Faceted Search style queries.

For example, we can group schools by city and count schools per group, giving us the number of schools per city. Or we could group by school class (independent/state) and see the average number of students per group.

In [ ]:
#Perform aggregation by city and count number of records
request = AggregateRequest(f'*').group_by('@city', reducers.count().alias('count'))
res = r.ft("idx:schools").aggregate(request)
display_ft_agg(res)



,city,count
0,Goudhurst,1
1,London,2
2,Oxford,1


In [ ]:
#Perform aggregation by city and count number of students
request = AggregateRequest(f'*').group_by('@city', reducers.sum('@students').alias('students_count'))
res = r.ft("idx:schools").aggregate(request)
display_ft_agg(res)

,city,students_count
0,Goudhurst,721
1,London,1794
2,Oxford,1200
